In [1]:
# install the open cv library
!pip install opencv-python

In [2]:
!pip install xgboost

In [3]:
# importing the packages
import pandas as pd
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [4]:
# there are approx 5 features present overall across 75000 images
features_dataframe = pd.read_csv("final_dataset_containing_5_extracted_features.csv")

In [5]:
features_dataframe.shape

(72986, 12)

In [6]:
# viewing the features_dataframe
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
0,00000.png,125,101.0,73.0,NaN,NaN,89,130.0,87.0,NaN,NaN,No
1,000004.png,127,75.0,NaN,NaN,NaN,90,87.0,NaN,NaN,NaN,Yes
2,00001.png,126,75.0,76.0,NaN,NaN,91,90.0,139.0,NaN,NaN,No
3,000014.png,130,77.0,NaN,NaN,NaN,89,91.0,NaN,NaN,NaN,Yes
4,000016.png,127,173.0,74.0,NaN,NaN,90,61.0,91.0,NaN,NaN,Yes


In [7]:
# performing shuffling on features_dataframe
features_dataframe = features_dataframe.sample(frac=1)
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
69952,50798.png,80,128.0,74.0,NaN,NaN,142,90.0,91.0,NaN,NaN,No
3881,011833.png,126,76.0,78.0,NaN,NaN,89,89.0,139.0,NaN,NaN,Yes
49547,29346.png,127,80.0,82.0,NaN,NaN,89,89.0,140.0,NaN,NaN,No
48792,28553.png,125,76.0,87.0,102.0,NaN,88,89.0,139.0,135.0,NaN,No
8527,025777.png,98,76.0,108.0,123.0,NaN,123,87.0,141.0,90.0,NaN,Yes


In [8]:
features_dataframe = features_dataframe.fillna(-1)

In [9]:
# normalizing the data
is_gan = features_dataframe["is_gan"] 
features_dataframe.drop(columns=["file_name", "is_gan"], inplace=True)

In [10]:
standard_scalar = StandardScaler()
features_dataframe = standard_scalar.fit_transform(features_dataframe)
features_dataframe = pd.DataFrame(features_dataframe)
features_dataframe

,0,1,2,3,4,5,6,7,8,9
0,-1.114860,1.090625,0.565679,-0.432465,-0.189726,2.979070,0.156418,0.607535,-0.420531,-0.183362
1,0.751776,-0.274241,0.645110,-0.432465,-0.189726,-0.249139,0.127754,1.393569,-0.420531,-0.183362
2,0.792355,-0.169251,0.724541,-0.432465,-0.189726,-0.249139,0.127754,1.409944,-0.420531,-0.183362
3,0.711197,-0.274241,0.823829,2.312241,-0.189726,-0.310048,0.127754,1.393569,2.628979,-0.183362
4,-0.384437,-0.274241,1.240842,2.871841,-0.189726,1.821787,0.070425,1.426320,1.619950,-0.183362
...,...,...,...,...,...,...,...,...,...,...
72981,-1.155439,0.906893,1.737286,-0.432465,-0.189726,-0.310048,0.127754,0.591159,-0.420531,-0.183362
72982,0.711197,-0.221746,0.803972,-0.432465,-0.189726,-0.310048,0.156418,1.409944,-0.420531,-0.183362
72983,-1.155439,0.985635,0.784114,-0.432465,-0.189726,2.918160,0.213747,0.542032,-0.420531,-0.183362
72984,0.792355,-0.247994,-0.923653,-0.432465,-0.189726,-0.310048,0.127754,-0.899031,-0.420531,-0.183362


In [11]:
features_dataframe["is_gan"] = is_gan
features_dataframe["is_gan"] = features_dataframe["is_gan"].map(dict(Yes=1, No=0))
features_dataframe

,0,1,2,3,4,5,6,7,8,9,is_gan
0,-1.114860,1.090625,0.565679,-0.432465,-0.189726,2.979070,0.156418,0.607535,-0.420531,-0.183362,0
1,0.751776,-0.274241,0.645110,-0.432465,-0.189726,-0.249139,0.127754,1.393569,-0.420531,-0.183362,1
2,0.792355,-0.169251,0.724541,-0.432465,-0.189726,-0.249139,0.127754,1.409944,-0.420531,-0.183362,0
3,0.711197,-0.274241,0.823829,2.312241,-0.189726,-0.310048,0.127754,1.393569,2.628979,-0.183362,1
4,-0.384437,-0.274241,1.240842,2.871841,-0.189726,1.821787,0.070425,1.426320,1.619950,-0.183362,1
...,...,...,...,...,...,...,...,...,...,...,...
72981,-1.155439,0.906893,1.737286,-0.432465,-0.189726,-0.310048,0.127754,0.591159,-0.420531,-0.183362,0
72982,0.711197,-0.221746,0.803972,-0.432465,-0.189726,-0.310048,0.156418,1.409944,-0.420531,-0.183362,0
72983,-1.155439,0.985635,0.784114,-0.432465,-0.189726,2.918160,0.213747,0.542032,-0.420531,-0.183362,0
72984,0.792355,-0.247994,-0.923653,-0.432465,-0.189726,-0.310048,0.127754,-0.899031,-0.420531,-0.183362,0


In [12]:
features_dataframe.isna().sum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
is_gan    0
dtype: int64

In [13]:
x = features_dataframe
y = features_dataframe["is_gan"]
x.drop(columns=["is_gan"], inplace=True)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

In [15]:
f"Shape of training data = {x_train.shape[0]} and testing data = {x_test.shape[0]}"

'Shape of training data = 51090 and testing data = 21896'

## Performing experiments using the following hyperparameters to find the best model

In [25]:
# Performing Kfold cross validation
adaboost_pipeline = Pipeline([("classifier", AdaBoostClassifier())])
param_grid = {'classifier__n_estimators' : list(range(1,100)),
             'classifier__learning_rate' : [0.1, 0.01, 0.001, 0.0001],
             'classifier__algorithm' : ['SAMME', 'SAMME.R']
             }

adaboost_pipeline

Pipeline(steps=[('classifier', AdaBoostClassifier())])

In [26]:
gridsearch_cv = GridSearchCV(adaboost_pipeline, param_grid=param_grid, cv=10, verbose=2, n_jobs=-1)

In [27]:
gridsearch_cv.fit(x_train, y_train)

Fitting 10 folds for each of 792 candidates, totalling 7920 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier', AdaBoostClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__algorithm': ['SAMME', 'SAMME.R'],
                         'classifier__learning_rate': [0.1, 0.01, 0.001,
                                                       0.0001],
                         'classifier__n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...]},
             verbose=2)

In [28]:
gridsearch_cv.best_params_

{'classifier__algorithm': 'SAMME',
 'classifier__learning_rate': 0.1,
 'classifier__n_estimators': 3}

## Training the model on the above best parameters

In [29]:
adaboost_model = AdaBoostClassifier(algorithm="SAMME", learning_rate=0.1, n_estimators=3)
adaboost_model.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME', learning_rate=0.1, n_estimators=3)

In [31]:
adaboost_model_predictions = adaboost_model.predict(x_test)

In [33]:
cm = metrics.confusion_matrix(y_test, adaboost_model_predictions)
print(f"The confusion matrix for Decision Tree Classifier is given below: \n {cm}")

The confusion matrix for Decision Tree Classifier is given below: 
 [[14908     0]
 [ 6988     0]]


In [34]:
abc_precision = precision_score(y_test, adaboost_model_predictions, average='weighted', labels=np.unique(adaboost_model_predictions))
abc_precision

0.6808549506759225

In [35]:
abc_recall = recall_score(y_test, adaboost_model_predictions, average='weighted', labels=np.unique(adaboost_model_predictions))
abc_recall

1.0

In [36]:
abc_f1 = f1_score(y_test, adaboost_model_predictions, average='weighted', labels=np.unique(adaboost_model_predictions))
abc_f1

0.8101293337680686

In [37]:
# storing the deci model
filename = 'adaboost_model_final.pkl'
pickle.dump(adaboost_model, open(filename, 'wb'))